In [ ]:
import numpy as np
import logging
import iris.coord_systems
import iris.fileformats
from iris.util import equalise_attributes
from iris.util import unify_time_units
import iris.quickplot as qplt
import iris.plot as iplt
import matplotlib.pyplot as plt
import datetime 
import iris.coord_categorisation
import os
import warnings
from bokeh.palettes import Sunset8, TolYlOrBr9
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, Select, \
    LinearColorMapper, ColorBar, CheckboxGroup, Button, GeoJSONDataSource,\
    BasicTicker, PrintfTickFormatter
import pandas as pd
from bokeh.models import WMTSTileSource
import xyzservices.providers as xyz
import math

warnings.filterwarnings("ignore")
%matplotlib tk

In [2]:
# Function to convert lat lon to Web Mercator coordinates
def lat_lon_to_web_mercator(lat, lon):
    x = lon * 20037508.34 / 180
    y = math.log(math.tan((90 + lat) * math.pi / 360)) / (math.pi / 180)
    y = y * 20037508.34 / 180
    return x, y

In [3]:
def prepare_calendar(cube):
    # Setting up the dates on data
    for coord_name, coord_func in [('year', iris.coord_categorisation.add_year),
                                   ('month_number', iris.coord_categorisation.add_month_number),
                                   ('day_of_month', iris.coord_categorisation.add_day_of_month),
                                   ('hour', iris.coord_categorisation.add_hour)]:
        if not cube.coords(coord_name):
            coord_func(cube, 'time', name=coord_name)
    return cube

def create_dates_dt(cube):
    cube = prepare_calendar(cube)
    cube_dates_dt = [datetime.datetime(y, m, d, h) for y, m, d, h in zip(cube.coord('year').points,
                                                                   cube.coord('month_number').points,
                                                                   cube.coord('day_of_month').points, 
                                                                   cube.coord('hour').points)]
    return cube_dates_dt

In [4]:
date = datetime.datetime(2024, 2, 19, 0)
ntimes_analysis = 332
ntimes_forecast = 28

In [5]:
analysis_cube =  iris.load_cube('/scratch/hadpx/SEA_monitoring/processed_SEA_data/analysis/eqwaves/precipitation_flux_analysis_20240219_00.nc')
analysis_cube *= 3600.
analysis_cube.coord('time')

<DimCoord: time / (hours since 1970-01-01 00:00:00)  [...]  shape(332,)>

In [90]:
forecast_files = ['/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T006.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T012.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T018.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T024.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T030.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T036.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T042.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T048.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T054.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T060.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T066.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T072.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T078.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T084.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T090.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T096.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T102.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T108.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T114.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T120.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T126.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T132.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T138.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T144.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T150.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T156.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T162.pp', '/scratch/hadpx/SEA_monitoring/raw_data/mogreps/eqwaves/20240219/00/000/qg00T168.pp']

In [91]:
forecast_cube = iris.load_cube(forecast_files, 'precipitation_amount')
forecast_cube.data[1:] -= forecast_cube.data[:-1]
forecast_cube = forecast_cube.regrid(analysis_cube, iris.analysis.Linear())
forecast_cube

<iris 'Cube' of precipitation_amount / (kg m-2) (-- : 28; latitude: 49; longitude: 360)>

In [110]:
print(analysis_cube.shape, forecast_cube.shape)
#datetime_list = sorted([date + datetime.timedelta(hours=(i+1) * 6)
#                                 for i in range(-1*ntimes_analysis, ntimes_forecast)])
analysis_datetime_list = sorted([date + datetime.timedelta(hours=(i+1)* 6)
                                 for i in range(-ntimes_analysis, 0)])

forecast_datetime_list = sorted([date + datetime.timedelta(hours=(i+1)* 6)
                                 for i in range(0, ntimes_forecast)])


all_dates = analysis_datetime_list+forecast_datetime_list
# Convert datetime values to Iris-compatible units
time_units = 'hours since 1970-01-01 00:00:00'
time_values = iris.util.cf_units.date2num(all_dates, time_units, calendar='gregorian')

# Create time coordinate
time_coord = iris.coords.DimCoord(time_values, standard_name='time', units=time_units)

#analysis_datetime_list[-10:]
#all_dates
all_dates[-29]

(332, 49, 360) (28, 49, 360)


datetime.datetime(2024, 2, 19, 0, 0)

In [111]:
concatenated_array = np.concatenate((analysis_cube.data, forecast_cube.data), axis=0)
ndims = len(concatenated_array.shape)
datetime_list = sorted([date + datetime.timedelta(hours=(i+1)* 6)
                                 for i in range(-1*ntimes_analysis, ntimes_forecast)])
# Convert datetime values to Iris-compatible units
time_units = 'hours since 1970-01-01 00:00:00'
time_values = iris.util.cf_units.date2num(datetime_list, time_units, calendar='gregorian')

# Create time coordinate
time_coord = iris.coords.DimCoord(time_values, standard_name='time', units=time_units)
# build cube
ndims = len(concatenated_array.shape)
if ndims == 4:
    concat_cube = iris.cube.Cube(
        concatenated_array,
        long_name=forecast_cube.long_name,
        units=forecast_cube.units,
        attributes=None,
        dim_coords_and_dims=[(time_coord, 0), (forecast_cube.coord('pressure'), 1),
                             (forecast_cube.coord('latitude'), 2),
                             (forecast_cube.coord('longitude'), 3)]
    )
elif ndims == 3:
    concat_cube = iris.cube.Cube(
        concatenated_array,
        long_name=forecast_cube.long_name,
        units=forecast_cube.units,
        attributes=None,
        dim_coords_and_dims=[(time_coord, 0),
                             (forecast_cube.coord('latitude'), 1),
                             (forecast_cube.coord('longitude'), 2)]
    )
print(concat_cube)

unknown / (kg m-2)                  (time: 360; latitude: 49; longitude: 360)
    Dimension coordinates:
        time                             x              -              -
        latitude                         -              x              -
        longitude                        -              -              x


In [69]:
dim_coords_and_dims = [(time_coord, 0)]
for coord in analysis_cube.dim_coords[1:]:
    dim_coords_and_dims.append((coord, analysis_cube.coord_dims(coord)[0]))
concat_cube = iris.cube.Cube(
            concatenated_array,
            long_name=forecast_cube.long_name,
            units=forecast_cube.units,
            attributes=None,
            dim_coords_and_dims=dim_coords_and_dims
        )
concat_cube

Unknown (kg m-2),time,latitude,longitude
Shape,360,49,360
Dimension coordinates,,,
time,x,-,-
latitude,-,x,-
longitude,-,-,x


In [10]:
mem_labels = [f'{fc:03}' for fc in range(0, 18)]
mem_labels

['000',
 '001',
 '002',
 '003',
 '004',
 '005',
 '006',
 '007',
 '008',
 '009',
 '010',
 '011',
 '012',
 '013',
 '014',
 '015',
 '016',
 '017']

In [12]:
outfile_dir = '/scratch/hadpx/SEA_monitoring/processed_SEA_data/mogreps/eqwaves/20240324_00'
date_label = '20240324_00'
precip_files = [os.path.join(outfile_dir, f'precipitation_flux_combined_{date_label}Z_{mem}.nc') for mem in mem_labels]
precip_files = [file for file in precip_files if os.path.exists(file)]
#precip_files

In [13]:
pr_cube = iris.load_cube(precip_files)

In [14]:
#pr_ens_mean = pr_cube.collapsed('realization', iris.analysis.MEAN)
times2plot = [t for t in range(-96, 174, 6)]
pressures = ['850', '200']
ntimes = len(times2plot)

pr_cube = pr_cube[:,-ntimes:]
# Assuming `times2plot` and `create_dates_dt(pr_cube)` are lists
data = [(i, l, d) for i, l, d in zip(range(ntimes), times2plot, create_dates_dt(pr_cube))]

# Creating DataFrame
df = pd.DataFrame(data, columns=['Index', 'Lead', 'Date'])
df.iloc[-28].Date.strftime('%Y%m%d %HZ')

'20240324 06Z'

In [15]:
thresholds = {'precip':5, 
              'Kelvin_850':-1*1e-6, 'Kelvin_200':-2*1e-6, 
              'WMRG_850':-1*1e-6, 'WMRG_200':-2*1e-6,
              'R1_850':5*1e-6, 'R1_200':2*1e-6, 
              'R2_850':2.5*1e-6, 'R2_200':2*1e-6}

In [44]:
wname = 'WMRG'
pressure_level = 850

shade_var = pr_cube.collapsed('realization', iris.analysis.PROPORTION,
                               function=lambda values: values > thresholds['precip'])
shade_cbar_title = f"Probability of precipitation >= {thresholds['precip']} mm day-1"

if wname in ['Kelvin', 'WMRG']:
    wave_files = [os.path.join(outfile_dir, f'div_wave_{wname}_{date_label}Z_{mem}.nc') for mem in mem_labels]
    wave_files = [file for file in wave_files if os.path.exists(file)]
    wave_variable = iris.load_cube(wave_files)
    wave_variable = wave_variable.extract(iris.Constraint(pressure=float(pressure_level)))
    wave_variable = wave_variable[:, -ntimes:]
    contour_var = wave_variable.collapsed('realization', iris.analysis.PROPORTION,
                               function=lambda values: values <= thresholds[f'{wname}_{pressure_level}'])
    contour_cbar_title = f"Probability of {wname} divergence <= {thresholds[f'{wname}_{pressure_level}']:0.1e} s-1"
elif wname in ['R1', 'R2']:
    wave_files = [os.path.join(outfile_dir, f'vort_wave_{wname}_{date_label}Z_{mem}.nc') for mem in mem_labels]
    wave_files = [file for file in wave_files if os.path.exists(file)]
    
    wave_variable = iris.load_cube(wave_files)
    wave_variable = wave_variable.extract(iris.Constraint(pressure=float(pressure_level)))
    wave_variable = wave_variable[:, -ntimes:]
    contour_var = wave_variable.collapsed('realization', iris.analysis.PROPORTION,
                               function=lambda values: values >= thresholds[f'{wname}_{pressure_level}'])
    contour_cbar_title = f"Probability of {wname} vorticity >= {thresholds[f'{wname}_{pressure_level}']:0.1e} s-1"
shade_var, shade_cbar_title, contour_cbar_title

(<iris 'Cube' of unknown / (1) (time: 45; latitude: 49; longitude: 360)>,
 'Probability of precipitation >= 5 mm day-1',
 'Probability of WMRG divergence <= -1.0e-06 s-1')

In [17]:
lead = 6
t = df.loc[df['Lead']==lead].Index.values[0]
date_string = df['Date'].loc[df['Lead']==lead].astype('O').tolist()[0].strftime('%Y/%m/%d %HZ')
if int(lead) < 0:
    figure_tite = f"{shade_cbar_title};  {contour_cbar_title} \nValid on {date_string} at T{lead}"
else:
    figure_tite = f"{shade_cbar_title};  {contour_cbar_title} \nValid on {date_string} at T+{lead}"
figure_tite

'Probability of precipitation >= 5 mm day-1;  Probability of WMRG divergence <= -1.0e-06 s-1 \nValid on 2024/03/24 06Z at T+6'

In [18]:
# Convert the numpy datetime64 object to a string in the desired format
#date_string = np.datetime_as_string(filtered_date, unit='h')
#date_string

print(t)

17


In [37]:
source = ColumnDataSource(data=dict(
    x=lons,
    y=lats,
    shade_var=shade_var,
))
source

ColumnDataSource(id='p2444', ...)

In [43]:
from bokeh.palettes import Iridescent23, TolYlOrBr9, Bokeh8, Greys9, Blues9
x_range = (0, 180)  # could be anything - e.g.(0,1)
y_range = (-24, 24)

# Plot setup
plot_width=1100
width = plot_width
aspect = (max(x_range) - min(x_range)) / (max(y_range) - min(y_range))
height = int(width / (0.6 * aspect))


plot = figure(height=height, width=width, x_range=x_range, y_range=y_range,
              tools=["pan, reset, save, wheel_zoom, hover"], 
              x_axis_label='Longitude', y_axis_label='Latitude', aspect_scale=4, 
              title=figure_tite)

# Specify the latitude and longitude range for the map
'''
min_lat, max_lat = -24, 24
min_lon, max_lon = 0, 180

# Convert the latitude and longitude range to Web Mercator coordinates
min_x, min_y = lat_lon_to_web_mercator(min_lat, min_lon)
max_x, max_y = lat_lon_to_web_mercator(max_lat, max_lon)

# range bounds supplied in web mercator coordinates
plot = figure(height=400, width=1600, x_range=(min_x, max_x), y_range=(min_y, max_y),
              tools=["pan, reset, save, wheel_zoom, hover"],
              x_axis_label='Longitude', y_axis_label='Latitude',
              x_axis_type="mercator", y_axis_type="mercator", aspect_scale=4)
plot.add_tile(xyz.OpenStreetMap.Mapnik)
'''

plot.title.text_font_size = "16pt"

shade_levels = np.arange(0.1, 1.1, 0.1)

color_mapper_z = LinearColorMapper(palette='Iridescent23', low=shade_levels.min(), high=shade_levels.max())
color_bar = ColorBar(color_mapper=color_mapper_z, major_label_text_font_size="12pt",
                     label_standoff=6, border_line_color=None, orientation="horizontal",
                     location=(0, 0), width=400, title=shade_cbar_title, title_text_font_size="14pt")

lons, lats = np.meshgrid(shade_var.coord('longitude').points, shade_var.coord('latitude').points)
shade_var = shade_var.data[t]
source = ColumnDataSource(data=dict(
    x=[lons],
    y=[lats],
    shade_var=[shade_var],
))
plot.image(image=[shade_var], x=0, y=-24,
           dw=360, dh=48, alpha=0.8,
           color_mapper=color_mapper_z)
# Define hover tooltips (replace 'shade_var' and 'hover_data' with your specific data and labels)
hover = plot.select_one(HoverTool)

TOOLTIPS = [
    ("(x,y)", "($x, $y)"),
    ("desc", "@shade_var"),
]
hover.tooltips = TOOLTIPS  # Example tooltip, customize as needed

plot.add_layout(color_bar, 'below')






contour_levels = np.arange(0.4, 1.2, 0.2)
lons, lats = np.meshgrid(contour_var.coord('longitude').points, contour_var.coord('latitude').points)
contour_renderer = plot.contour(lons, lats, contour_var.data[t], contour_levels, fill_color=None, fill_alpha=0.3, 
                                line_color=Bokeh8, line_alpha=0.5, line_width=5)
colorbar = contour_renderer.construct_color_bar(major_label_text_font_size="12pt", 
                                                orientation="horizontal", location=(-500, -135), width=400,
                                                title=contour_cbar_title, title_text_font_size="14pt")
plot.add_layout(colorbar, "right")

with open("custom.geo.json", "r") as f:
    countries = GeoJSONDataSource(geojson=f.read())

plot.patches("xs", "ys", color=None, line_color="grey", source=countries, alpha=0.75)

show(plot)

In [17]:
ens_mean = kelvin_div_cubes.collapsed('realization', iris.analysis.MEAN)

In [40]:

#precip_prob

In [30]:
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap, show

map_options = GMapOptions(lat=0., lng=100.7394, map_type="roadmap", zoom=2)

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:
p = gmap("GOOGLE_API_KEY", map_options, title="Austin")

source = ColumnDataSource(
    data=dict(lat=[ 30.29,  30.20,  30.29],
              lon=[-97.70, -97.74, -97.78]),
)

p.scatter(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, source=source)

show(p)

In [56]:
qplt.contourf(kelvin_div_cubes[:,t, 0].collapsed('realization', iris.analysis.PROPORTION,
                               function=lambda values: values < -2*1e-6))

In [75]:
lons, lats = np.meshgrid(precip_prob.coord('longitude').points, precip_prob.coord('latitude').points)
precip_prob.shape, lats.shape

((360, 49, 360), (49, 360))

In [ ]:
import tqdm

Web Mercator X: -111319.49079326664, Web Mercator Y: 0.0


In [139]:
countries_data['features'][0]['geometry'][]

{'type': 'Polygon',
 'coordinates': [[[61.210817, 35.650072],
   [62.230651, 35.270664],
   [62.984662, 35.404041],
   [63.193538, 35.857166],
   [63.982896, 36.007957],
   [64.546479, 36.312073],
   [64.746105, 37.111818],
   [65.588948, 37.305217],
   [65.745631, 37.661164],
   [66.217385, 37.39379],
   [66.518607, 37.362784],
   [67.075782, 37.356144],
   [67.83, 37.144994],
   [68.135562, 37.023115],
   [68.859446, 37.344336],
   [69.196273, 37.151144],
   [69.518785, 37.608997],
   [70.116578, 37.588223],
   [70.270574, 37.735165],
   [70.376304, 38.138396],
   [70.806821, 38.486282],
   [71.348131, 38.258905],
   [71.239404, 37.953265],
   [71.541918, 37.905774],
   [71.448693, 37.065645],
   [71.844638, 36.738171],
   [72.193041, 36.948288],
   [72.63689, 37.047558],
   [73.260056, 37.495257],
   [73.948696, 37.421566],
   [74.980002, 37.41999],
   [75.158028, 37.133031],
   [74.575893, 37.020841],
   [74.067552, 36.836176],
   [72.920025, 36.720007],
   [71.846292, 36.509942],
